In [26]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

weight = torch.load("/home/hyunho/sfda/swin_tiny_patch4_window7_224.pth", map_location= device)

/tmp/ipykernel_6308/1653038833.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  weight = torch.load("/home/hyunho/sfda/swin_tiny_patch4_window7_224.pth", map_location= de

In [27]:
len(weight["model"].keys())

190

In [31]:
for i in weight["model"].keys():
  print(i)

patch_embed.proj.weight
patch_embed.proj.bias
patch_embed.norm.weight
patch_embed.norm.bias
layers.0.blocks.0.norm1.weight
layers.0.blocks.0.norm1.bias
layers.0.blocks.0.attn.qkv.weight
layers.0.blocks.0.attn.qkv.bias
layers.0.blocks.0.attn.proj.weight
layers.0.blocks.0.attn.proj.bias
layers.0.blocks.0.norm2.weight
layers.0.blocks.0.norm2.bias
layers.0.blocks.0.mlp.fc1.weight
layers.0.blocks.0.mlp.fc1.bias
layers.0.blocks.0.mlp.fc2.weight
layers.0.blocks.0.mlp.fc2.bias
layers.0.blocks.1.norm1.weight
layers.0.blocks.1.norm1.bias
layers.0.blocks.1.attn.qkv.weight
layers.0.blocks.1.attn.qkv.bias
layers.0.blocks.1.attn.proj.weight
layers.0.blocks.1.attn.proj.bias
layers.0.blocks.1.norm2.weight
layers.0.blocks.1.norm2.bias
layers.0.blocks.1.mlp.fc1.weight
layers.0.blocks.1.mlp.fc1.bias
layers.0.blocks.1.mlp.fc2.weight
layers.0.blocks.1.mlp.fc2.bias
layers.0.downsample.norm.weight
layers.0.downsample.norm.bias
layers.1.blocks.0.norm1.weight
layers.1.blocks.0.norm1.bias
layers.1.blocks.0.attn

In [75]:
from transformers import Mask2FormerConfig, Mask2FormerModel, Mask2FormerForUniversalSegmentation, AutoImageProcessor
cfg = Mask2FormerConfig()
cfg.backbone_config.depths = [2, 2, 6, 2]
model = Mask2FormerForUniversalSegmentation(cfg).to(device)


In [80]:
model.model.transformer_module


Mask2FormerTransformerModule(
  (position_embedder): Mask2FormerSinePositionEmbedding()
  (queries_embedder): Embedding(100, 256)
  (queries_features): Embedding(100, 256)
  (decoder): Mask2FormerMaskedAttentionDecoder(
    (layers): ModuleList(
      (0-8): 9 x Mask2FormerMaskedAttentionDecoderLayer(
        (self_attn): Mask2FormerAttention(
          (k_proj): Linear(in_features=256, out_features=256, bias=True)
          (v_proj): Linear(in_features=256, out_features=256, bias=True)
          (q_proj): Linear(in_features=256, out_features=256, bias=True)
          (out_proj): Linear(in_features=256, out_features=256, bias=True)
        )
        (activation_fn): ReLU()
        (self_attn_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (cross_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
        )
        (cross_attn_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine

In [41]:
len(model.model.pixel_level_module.encoder.state_dict())

237

In [40]:
for i in model.model.pixel_level_module.encoder.state_dict():
  print(i)

embeddings.patch_embeddings.projection.weight
embeddings.patch_embeddings.projection.bias
embeddings.norm.weight
embeddings.norm.bias
encoder.layers.0.blocks.0.layernorm_before.weight
encoder.layers.0.blocks.0.layernorm_before.bias
encoder.layers.0.blocks.0.attention.self.relative_position_bias_table
encoder.layers.0.blocks.0.attention.self.relative_position_index
encoder.layers.0.blocks.0.attention.self.query.weight
encoder.layers.0.blocks.0.attention.self.query.bias
encoder.layers.0.blocks.0.attention.self.key.weight
encoder.layers.0.blocks.0.attention.self.key.bias
encoder.layers.0.blocks.0.attention.self.value.weight
encoder.layers.0.blocks.0.attention.self.value.bias
encoder.layers.0.blocks.0.attention.output.dense.weight
encoder.layers.0.blocks.0.attention.output.dense.bias
encoder.layers.0.blocks.0.layernorm_after.weight
encoder.layers.0.blocks.0.layernorm_after.bias
encoder.layers.0.blocks.0.intermediate.dense.weight
encoder.layers.0.blocks.0.intermediate.dense.bias
encoder.lay

In [52]:
print("query : ", model.model.pixel_level_module.encoder.encoder.layers[0].blocks[0].attention.self.query.weight.shape)
print("key : ", model.model.pixel_level_module.encoder.encoder.layers[0].blocks[0].attention.self.key.weight.shape)
print("value : ", model.model.pixel_level_module.encoder.encoder.layers[0].blocks[0].attention.self.value.weight.shape)

query :  torch.Size([96, 96])
key :  torch.Size([96, 96])
value :  torch.Size([96, 96])


In [61]:
model.model.pixel_level_module.encoder.hidden_states_norms.stage1.weight.shape

torch.Size([96])

In [67]:
model.model.pixel_level_module

Mask2FormerPixelLevelModule(
  (encoder): SwinBackbone(
    (embeddings): SwinEmbeddings(
      (patch_embeddings): SwinPatchEmbeddings(
        (projection): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): SwinEncoder(
      (layers): ModuleList(
        (0): SwinStage(
          (blocks): ModuleList(
            (0-1): 2 x SwinLayer(
              (layernorm_before): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
              (attention): SwinAttention(
                (self): SwinSelfAttention(
                  (query): Linear(in_features=96, out_features=96, bias=True)
                  (key): Linear(in_features=96, out_features=96, bias=True)
                  (value): Linear(in_features=96, out_features=96, bias=True)
                  (dropout): Dropout(p=0.0, inplace=False)
                )
                (output): SwinSelfOut

In [65]:
96*4

384

In [68]:
print("norm weight : ",weight["model"]["norm.weight"].shape)
print("norm bias : ",weight["model"]["norm.bias"].shape)
print("head weight : ",weight["model"]["head.weight"].shape)
print("head bias : ",weight["model"]["head.bias"].shape)



norm weight :  torch.Size([768])
norm bias :  torch.Size([768])
head weight :  torch.Size([1000, 768])
head bias :  torch.Size([1000])


In [42]:
mseg_weight = torch.load("/home/hyunho/sfda/swin_tiny_patch4_window7_224_20220317-1cdeb081.pth", map_location=device)

/tmp/ipykernel_6308/2471000216.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  mseg_weight = torch.load("/home/hyunho/sfda/swin_tiny_patch4_window7_224_20220317-1cdeb081

In [44]:
len(mseg_weight.keys())

188

In [55]:
selected_layers = [x for x in model.state_dict().keys() if "pixel_level_module.encoder" in x]

In [57]:
for i in selected_layers:
  print(i)

model.pixel_level_module.encoder.embeddings.patch_embeddings.projection.weight
model.pixel_level_module.encoder.embeddings.patch_embeddings.projection.bias
model.pixel_level_module.encoder.embeddings.norm.weight
model.pixel_level_module.encoder.embeddings.norm.bias
model.pixel_level_module.encoder.encoder.layers.0.blocks.0.layernorm_before.weight
model.pixel_level_module.encoder.encoder.layers.0.blocks.0.layernorm_before.bias
model.pixel_level_module.encoder.encoder.layers.0.blocks.0.attention.self.relative_position_bias_table
model.pixel_level_module.encoder.encoder.layers.0.blocks.0.attention.self.relative_position_index
model.pixel_level_module.encoder.encoder.layers.0.blocks.0.attention.self.query.weight
model.pixel_level_module.encoder.encoder.layers.0.blocks.0.attention.self.query.bias
model.pixel_level_module.encoder.encoder.layers.0.blocks.0.attention.self.key.weight
model.pixel_level_module.encoder.encoder.layers.0.blocks.0.attention.self.key.bias
model.pixel_level_module.enco

In [69]:
mmseg = torch.load('/home/hyunho/sfda/swin_tiny_patch4_window7_224_20220317-1cdeb081.pth', map_location=device)

/tmp/ipykernel_6308/14298589.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  mmseg = torch.load('/home/hyunho/sfda/swin_tiny_patch4_window7_224_20220317-1cdeb081.pth', m

In [72]:
for i in mmseg.keys():
  print(i)

patch_embed.projection.weight
patch_embed.projection.bias
patch_embed.norm.weight
patch_embed.norm.bias
stages.0.blocks.0.norm1.weight
stages.0.blocks.0.norm1.bias
stages.0.blocks.0.attn.w_msa.qkv.weight
stages.0.blocks.0.attn.w_msa.qkv.bias
stages.0.blocks.0.attn.w_msa.proj.weight
stages.0.blocks.0.attn.w_msa.proj.bias
stages.0.blocks.0.norm2.weight
stages.0.blocks.0.norm2.bias
stages.0.blocks.0.ffn.layers.0.0.weight
stages.0.blocks.0.ffn.layers.0.0.bias
stages.0.blocks.0.ffn.layers.1.weight
stages.0.blocks.0.ffn.layers.1.bias
stages.0.blocks.1.norm1.weight
stages.0.blocks.1.norm1.bias
stages.0.blocks.1.attn.w_msa.qkv.weight
stages.0.blocks.1.attn.w_msa.qkv.bias
stages.0.blocks.1.attn.w_msa.proj.weight
stages.0.blocks.1.attn.w_msa.proj.bias
stages.0.blocks.1.norm2.weight
stages.0.blocks.1.norm2.bias
stages.0.blocks.1.ffn.layers.0.0.weight
stages.0.blocks.1.ffn.layers.0.0.bias
stages.0.blocks.1.ffn.layers.1.weight
stages.0.blocks.1.ffn.layers.1.bias
stages.0.downsample.norm.weight
stag

In [74]:
cfg

Mask2FormerConfig {
  "activation_function": "relu",
  "backbone": null,
  "backbone_config": {
    "depths": [
      2,
      2,
      6,
      2
    ],
    "drop_path_rate": 0.3,
    "in_channels": 3,
    "model_type": "swin",
    "out_features": [
      "stage1",
      "stage2",
      "stage3",
      "stage4"
    ],
    "out_indices": [
      1,
      2,
      3,
      4
    ]
  },
  "backbone_kwargs": null,
  "class_weight": 2.0,
  "common_stride": 4,
  "decoder_layers": 10,
  "dice_weight": 5.0,
  "dim_feedforward": 2048,
  "dropout": 0.0,
  "encoder_feedforward_dim": 1024,
  "encoder_layers": 6,
  "enforce_input_projection": false,
  "feature_size": 256,
  "feature_strides": [
    4,
    8,
    16,
    32
  ],
  "hidden_dim": 256,
  "ignore_value": 255,
  "importance_sample_ratio": 0.75,
  "init_std": 0.02,
  "init_xavier_std": 1.0,
  "mask_feature_size": 256,
  "mask_weight": 5.0,
  "model_type": "mask2former",
  "no_object_weight": 0.1,
  "num_attention_heads": 8,
  "num_hidden

In [81]:
import evaluate

metric = evaluate.load("mean_iou")

AttributeError: 'DownloadConfig' object has no attribute 'token'